In [128]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from string import punctuation
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import Counter

import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer

import re
import os
import email
from email.parser import BytesParser
from bs4 import BeautifulSoup

In [129]:
def extract_text_from_eml(eml_path):
    with open(eml_path, 'rb') as eml_file:
        msg = BytesParser().parse(eml_file)
        text = ''
        for part in msg.walk():
            if part.get_content_type() == 'text/plain':
                text += part.get_payload(decode=True).decode(part.get_content_charset(), 'ignore')
            elif part.get_content_type() == 'text/html':
                text += BeautifulSoup(part.get_payload(decode=True), 'html.parser').get_text()
    return text

def clean_text(text):
    
    return text.strip().replace("\n"," ").replace("\xa0"," ")

def process_eml_files(eml_folder):
    eml_files = [f for f in os.listdir(eml_folder) if f.endswith('.eml')]
    data = []
    for eml_file in eml_files:
        eml_path = os.path.join(eml_folder, eml_file)
        text = extract_text_from_eml(eml_path)
        cleaned_text = clean_text(text)
        data.append({'File': eml_file, 'Text': cleaned_text})
    df = pd.DataFrame(data)
    return df

# Укажите путь к папке с файлами .eml
eml_folder = 'C:/Users/Нур/Spam classification/data'
df = process_eml_files(eml_folder)
print(df.head())


    File                                               Text
0  1.eml  View email in your browser  Dear authors and r...
1  2.eml  View email in your browser  [ https://t.me/NJD...
2  3.eml  РџРёСЃСЊРјРѕ                                  ...


In [132]:
df.iloc[2,1]

'РџРёСЃСЊРјРѕ                                          Дайджест проектов hh.kz     Обновления и новости проектов, которые помогают в успешном трудоустройстве                                                                Не белые воротнички: как успешно пройти собеседование тем, кто не хочет трудиться в офисе   Как проходить собеседование офисным служащим, написано много. Но, если ваша профессия в прямом смысле слова в руках, включаются иные механизмы. Хотя задача та же — произвести впечатление и получить работу. В этой статье мы даём несколько ценных советов, которые помогут достичь цели.                                                          «Почему вы ушли с прежней работы?» — как не стоит отвечать на этот вопрос    Этот вопрос — один из самых нелюбимых у соискателей и самых любимых у рекрутеров. В вашем ответе важна не только суть (причина увольнения), но и то, как вы эту суть преподнесете. Ответ показывает вашу мотивацию, умение выходить достойно из непростой ситуации и демонстр

In [131]:
df

,File,Text
0,1.eml,View email in your browser Dear authors and r...
1,2.eml,View email in your browser [ https://t.me/NJD...
2,3.eml,РџРёСЃСЊРјРѕ ...


In [6]:
from transformers import pipeline

# Создание пайплайна для перевода с английского на русский
# В этом случае не нужно указывать конкретную модель
translator = pipeline("translation", model="facebook/m2m100_418M", src_lang="en", tgt_lang="ru")

# Пример текста для перевода
text = "Hello, world! This is an example of text translation."

# Перевод текста
translated_text = translator(text, max_length=512)

# Вывод переведенного текста
print(translated_text[0]['translation_text'])


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

C:\Users\Нур\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Нур\.cache\huggingface\hub\models--facebook--m2m100_418M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

KeyboardInterrupt: 